# Hack and Learn Team 22 - Graphs

You'll need the SQLite database of data.

You probably won't be able to upload it directly to the notebook - it will time out. Instead upload it to your Google Drive, then mount the drive.



## First set up the database connection & graph library



In [ ]:
!pip install plotly kaleido


In [ ]:
import sqlite3
import plotly.express

sqlite_filename = 'drive/MyDrive/HackAndLearnTeam22/output.sqlite'

connection = sqlite3.connect(sqlite_filename)
connection.row_factory = sqlite3.Row
cursor = connection.cursor()

## Average hhasset_total by treatment_simulated



In [ ]:
sql = 'select treatment_simulated, avg(hhasset_total) AS hhasset_total from data group by treatment_simulated';
cursor.execute(sql)
data = cursor.fetchall()

viz = plotly.express.bar(data)
viz.show()
viz.write_image("average-hhasset_total-by-treatment_simulated.png")

## Distribution Curve hhasset_total by treatment_simulated


In [ ]:
# work out buckets
sql = 'select max(cast(hhasset_total as int)) AS max, min(cast(hhasset_total as int)) AS min  from data ';
cursor.execute(sql)
data = cursor.fetchone()
max = data['max']
min = data['min']
number_of_buckets = 1000

buckets = []
for i in range(0, number_of_buckets):
    buckets.append({
        'min': int(min + (max -min)/number_of_buckets*i),
        'max': int(min + (max -min)/number_of_buckets*(i+1)),
    })

# Get data
for bucket in buckets:
    sql = 'select count(*) AS c from data where hhasset_total > ? AND hhasset_total <= ? AND treatment_simulated=0  ';
    cursor.execute(sql, (bucket['min'], bucket['max']))
    data = cursor.fetchone()
    bucket['treatment_simulated_0'] = data['c']

    sql = 'select count(*) AS c from data where hhasset_total > ? AND hhasset_total <= ? AND treatment_simulated=1  ';
    cursor.execute(sql, (bucket['min'], bucket['max']))
    data = cursor.fetchone()
    bucket['treatment_simulated_1'] = data['c']

#[print(i) for i in buckets]

# Viz
viz_data = [{'x':b['min'], 'y': b['treatment_simulated_0'], 'treatment_simulated':0} for b in buckets] + \
    [{'x':b['min'], 'y': b['treatment_simulated_1'], 'treatment_simulated':1} for b in buckets]
viz = plotly.express.histogram(viz_data, x='x', y='y', barmode='group', color='treatment_simulated', height=600, nbins=50)

viz.show()
viz.write_image("distribution-curve-hhasset_total-by-treatment_simulated.png")